In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/ViT-MaskedAttention'

/content/drive/.shortcut-targets-by-id/19BEgVkLwJC08ZvXW8kbzitRp0_uGrU0x/ViT-MaskedAttention


# Install pytorch-lightning

In [ ]:
!pip install -q pytorch-lightning
!pip install -q omegaconf
!pip install -q transformers

In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/19BEgVkLwJC08ZvXW8kbzitRp0_uGrU0x/ViT-MaskedAttention


# Import Lib

In [ ]:
import os
import random
import sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torch.utils.data as data_utils
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.models as models
from torchsummary import summary

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.lr_monitor import LearningRateMonitor

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score

# Data Module


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img = self.df.iloc[idx]['pixels']
        label = self.df.iloc[idx]['emotion']
        
        # Convert the numpy array to a PyTorch tensor
        img_tensor = torch.from_numpy(img).float().reshape(1, 48, 48)
        img_tensor = torch.cat([img_tensor, img_tensor, img_tensor], dim=0)
        
        if self.transform:
            img_tensor = self.transform(img_tensor)
            
        return img_tensor, label

In [ ]:
import torch
import pytorch_lightning as pl
import torchvision.datasets as datasets
import torchvision.transforms as transforms

mean = [0.4914, 0.4822, 0.4465]
std = [0.2470, 0.2435, 0.2616]

class FERDataModule(pl.LightningDataModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.data_dir = self.cfg.data.data_dir
        self.batch_size = self.cfg.data.batch_size
        self.num_workers = self.cfg.data.num_workers
        # Define data transformation pipeline with data augmentation
        self.transform_train = transforms.Compose([
            # transforms.ToPILImage(),
            # transforms.ToTensor(),
            # transforms.Resize(256),
            # transforms.RandomCrop(224),
            # transforms.RandomGrayscale(p=1),
            # transforms.RandomHorizontalFlip(),
            # transforms.RandomRotation(15),
            # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            # transforms.RandomGrayscale(p=0.1),
            transforms.Normalize(mean, std)
        ])

        self.transform_test = transforms.Compose([
            # transforms.ToTensor(),
            # transforms.Resize(256),
            # transforms.CenterCrop(224),
            transforms.Normalize(mean, std)
        ])

    def prepare_data(self):
        pass

    def setup(self, stage=None):
        # Assign train/val datasets for use in dataloaders
        if stage == 'fit' or stage is None:
            data = pd.read_csv(os.path.join(cfg.data.data_dir, 'train.csv'))
            data['pixels'] = data['pixels'].apply(lambda x : np.array(list(map(int, x.split(' ')))).reshape(48, 48))
            self.trainset, self.valset = train_test_split(data, train_size=0.7, random_state=42, stratify=data['emotion'], shuffle=True)

            self.train_dataset = ImageDataset(self.trainset, transform=self.transform_train)
            self.val_dataset = ImageDataset(self.valset, transform=self.transform_train)

        # Assign test dataset for use in dataloader(s)
        if stage == 'test' or stage is None:
            self.testset = pd.read_csv(os.path.join(cfg.data.data_dir, 'test.csv'))
            self.testset['pixels'] = self.testset['pixels'].apply(lambda x : np.array(list(map(int, x.split(' ')))).reshape(48, 48))

            self.test_dataset = ImageDataset(self.testset, transform=self.transform_test)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers
        )

# Model


In [ ]:
import torch
import numpy as np


def get_2d_sincos_pos_embed(embed_dim, grid_size, cls_token=False):
    """
    grid_size: int of the grid height and width
    return:
    pos_embed: [grid_size*grid_size, embed_dim] or [1+grid_size*grid_size, embed_dim] (w/ or w/o cls_token)
    """
    grid_h = np.arange(grid_size, dtype=np.float32)
    grid_w = np.arange(grid_size, dtype=np.float32)
    grid = np.meshgrid(grid_w, grid_h)  # here w goes first
    grid = np.stack(grid, axis=0)

    grid = grid.reshape([2, 1, grid_size, grid_size])
    pos_embed = get_2d_sincos_pos_embed_from_grid(embed_dim, grid)
    if cls_token:
        pos_embed = np.concatenate([np.zeros([1, embed_dim]), pos_embed], axis=0)
    return pos_embed


def get_2d_sincos_pos_embed_from_grid(embed_dim, grid):
    assert embed_dim % 2 == 0

    # use half of dimensions to encode grid_h
    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[0])  # (H*W, D/2)
    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[1])  # (H*W, D/2)

    emb = np.concatenate([emb_h, emb_w], axis=1)  # (H*W, D)
    return emb


def get_1d_sincos_pos_embed_from_grid(embed_dim, pos):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M,)
    out: (M, D)
    """
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=np.float)
    omega /= embed_dim / 2.0
    omega = 1.0 / 10000**omega  # (D/2,)

    pos = pos.reshape(-1)  # (M,)
    out = np.einsum("m,d->md", pos, omega)  # (M, D/2), outer product

    emb_sin = np.sin(out)  # (M, D/2)
    emb_cos = np.cos(out)  # (M, D/2)

    emb = np.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)
    return emb


def interpolate_pos_embed(model, checkpoint_model):
    if "pos_embed" in checkpoint_model:
        pos_embed_checkpoint = checkpoint_model["pos_embed"]
        embedding_size = pos_embed_checkpoint.shape[-1]
        num_patches = model.patch_embed.num_patches
        num_extra_tokens = model.pos_embed.shape[-2] - num_patches
        # height (== width) for the checkpoint position embedding
        orig_size = int((pos_embed_checkpoint.shape[-2] - num_extra_tokens) ** 0.5)
        # height (== width) for the new position embedding
        new_size = int(num_patches**0.5)
        # class_token and dist_token are kept unchanged
        if orig_size != new_size:
            print(
                "Position interpolate from %dx%d to %dx%d"
                % (orig_size, orig_size, new_size, new_size)
            )
            extra_tokens = pos_embed_checkpoint[:, :num_extra_tokens]
            # only the position tokens are interpolated
            pos_tokens = pos_embed_checkpoint[:, num_extra_tokens:]
            pos_tokens = pos_tokens.reshape(
                -1, orig_size, orig_size, embedding_size
            ).permute(0, 3, 1, 2)
            pos_tokens = torch.nn.functional.interpolate(
                pos_tokens,
                size=(new_size, new_size),
                mode="bicubic",
                align_corners=False,
            )
            pos_tokens = pos_tokens.permute(0, 2, 3, 1).flatten(1, 2)
            new_pos_embed = torch.cat((extra_tokens, pos_tokens), dim=1)
            checkpoint_model["pos_embed"] = new_pos_embed

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PatchEmbedding(nn.Module):
    def __init__(
        self,
        img_size,
        patch_size,
        in_channels=3,
        embedding_dim=768,
    ):
        super(PatchEmbedding, self).__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2

        self.projection_layer = nn.Conv2d(
            in_channels=in_channels,
            out_channels=embedding_dim,
            kernel_size=patch_size,
            stride=patch_size,
        )

    def forward(self, x):
        """Forward pass of patch embedding layer.

        Args:
            x (torch.Tensor): input image tensor, shape (B, C, H, W)
        """

        x = self.projection_layer(
            x
        )  # (batch_size, embedding_dim, n_patches ** 0.5, n_patches ** 0.5)
        x = x.flatten(2)  # (batch_size, embedding_dim, n_patches)
        x = x.transpose(1, 2)  # (batch_size, n_patches, embedding_dim)

        return x

    def get_num_patches(self):
        return self.n_patches


class Attention(nn.Module):
    def __init__(
        self,
        d_model,
        n_heads,
        qkv_bias=False,
        attn_p=0.0,
        proj_p=0.0,
    ):
        super(Attention, self).__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        self.scale = self.head_dim**-0.5

        self.qkv_linear = nn.Linear(d_model, d_model * 3, bias=qkv_bias)
        self.attn_dropout = nn.Dropout(attn_p)

        self.proj_linear = nn.Linear(d_model, d_model)
        self.proj_dropout = nn.Dropout(proj_p)

    def forward(self, x, q_mask=None):
        n_samples, n_patches, d_model = x.shape

        if d_model != self.d_model:
            raise ValueError(
                f"Input dimension {d_model} does not match layer dimension {self.d_model}"
            )

        qkv = self.qkv_linear(x)
        qkv = qkv.reshape(
            n_samples,
            n_patches,
            3,
            self.n_heads,
            self.head_dim,
        )  # (batch_size, n_patches, 3, n_heads, head_dim)
        qkv = qkv.permute(
            2, 0, 3, 1, 4
        )  # (3, batch_size, n_heads, n_patches, head_dim)
        q, k, v = qkv[0], qkv[1], qkv[2]  # (batch_size, n_heads, n_patches, head_dim)
        # print(f"q.shape: {q.shape}")

        dp = (
            q @ k.transpose(-2, -1)
        ) * self.scale  # (batch_size, n_heads, n_patches, n_patches)
        # print(f"dp.shape: {dp.shape}")

        if q_mask is not None:
            dp = dp.masked_fill(q_mask == 0, float("-inf"))

        attn = dp.softmax(dim=-1)
        attn = self.attn_dropout(attn)

        weighted_avg = attn @ v  # (batch_size, n_heads, n_patches, head_dim)
        weighted_avg = weighted_avg.transpose(
            1, 2
        )  # (batch_size, n_patches, n_heads, head_dim)
        weighted_avg = weighted_avg.flatten(2)  # (batch_size, n_patches, d_model)

        x = self.proj_linear(weighted_avg)
        x = self.proj_dropout(x)

        return x


class MLP(nn.Module):
    def __init__(
        self,
        in_features,
        hidden_features,
        out_features,
        p=0.0,
    ):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.dropout = nn.Dropout(p)
        self.gelu = nn.GELU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x


class Block(nn.Module):
    def __init__(
        self,
        d_model,
        n_heads,
        mlp_ratio=4.0,
        qkv_bias=True,
        attn_p=0.0,
        proj_p=0.0,
    ):
        super(Block, self).__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.mlp_ratio = mlp_ratio
        self.mlp_hidden_features = int(d_model * mlp_ratio)

        self.norm1 = nn.LayerNorm(d_model)
        self.attn = Attention(
            d_model=d_model,
            n_heads=n_heads,
            qkv_bias=qkv_bias,
            attn_p=attn_p,
            proj_p=proj_p,
        )

        self.norm2 = nn.LayerNorm(d_model)
        self.mlp = MLP(
            in_features=d_model,
            hidden_features=self.mlp_hidden_features,
            out_features=d_model,
        )

    def forward(self, x, q_mask=None):
        x = x + self.attn(self.norm1(x), q_mask=q_mask)
        x = x + self.mlp(self.norm2(x))

        return x


class ViT(nn.Module):
    def __init__(
        self,
        img_size=224,
        patch_size=16,
        in_channels=3,
        num_classes=1000,
        embedding_dim=768,
        depth=12,
        n_heads=12,
        mlp_ratio=4.0,
        qkv_bias=True,
        attn_p=0.0,
        p=0.0,
        fine_tune=False,
    ):
        super(ViT, self).__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.embedding_dim = embedding_dim
        self.depth = depth

        # input
        self.patch_embed = PatchEmbedding(
            img_size=img_size,
            patch_size=patch_size,
            in_channels=in_channels,
            embedding_dim=embedding_dim,
        )
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embedding_dim))
        self.pos_embed = nn.Parameter(
            torch.zeros(1, (img_size // patch_size) ** 2 + 1, embedding_dim)
        )
        self.pos_dropout = nn.Dropout(p=p)

        # transformer
        self.blocks = nn.ModuleList(
            [
                Block(
                    d_model=embedding_dim,
                    n_heads=n_heads,
                    mlp_ratio=mlp_ratio,
                    qkv_bias=qkv_bias,
                    attn_p=attn_p,
                    proj_p=p,
                )
                for _ in range(depth)
            ]
        )

        self.norm = nn.LayerNorm(embedding_dim)
        self.head = nn.Linear(embedding_dim, num_classes)

    def forward(self, x, q_mask=None, get_features=False):
        batch_size = x.shape[0]
        x = self.patch_embed(x)

        # add cls token and position embedding
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        # print(f"cls_tokens.shape: {cls_tokens.shape}")
        # print(f"x.shape: {x.shape}")
        x = torch.cat((cls_tokens, x), dim=1)
        if q_mask is not None:
            q_mask = F.pad(q_mask, (1, 0), value=True)
        x = x + self.pos_embed
        x = self.pos_dropout(x)

        # transformer
        for block in self.blocks:
            x = block(x, q_mask=q_mask)

        transformer_features = self.norm(x)

        # classification head
        x = self.head(transformer_features[:, 0])

        if get_features:
            return x, transformer_features
        return x


class MaskedAutoencoderViT(nn.Module):
    """Masked Autoencoder with VisionTransformer backbone"""

    def __init__(
        self,
        img_size=224,
        patch_size=16,
        in_channels=3,
        num_classes=1000,
        embedding_dim=1024,
        depth=24,
        n_heads=16,
        decoder_embed_dim=512,
        decoder_depth=8,
        decoder_num_heads=16,
        mlp_ratio=4.0,
        qkv_bias=True,
        norm_layer=nn.LayerNorm,
        norm_pix_loss=False,
        attn_p=0.0,
        p=0.0,
    ):
        super().__init__()

        # --------------------------------------------------------------------------
        # MAE encoder specifics
        self.patch_embed = PatchEmbedding(
            img_size, patch_size, in_channels, embedding_dim
        )
        num_patches = self.patch_embed.get_num_patches()

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embedding_dim))
        self.pos_embed = nn.Parameter(
            torch.zeros(1, num_patches + 1, embedding_dim), requires_grad=False
        )  # fixed sin-cos embedding

        self.blocks = nn.ModuleList(
            [
                Block(
                    embedding_dim,
                    n_heads,
                    mlp_ratio,
                    qkv_bias=qkv_bias,
                    attn_p=attn_p,
                    proj_p=p,
                )
                for _ in range(depth)
            ]
        )
        self.norm = norm_layer(embedding_dim)
        self.cls_head = nn.Linear(embedding_dim, num_classes)  # classification head
        # --------------------------------------------------------------------------

        # --------------------------------------------------------------------------
        # MAE decoder specifics
        self.decoder_embed = nn.Linear(embedding_dim, decoder_embed_dim, bias=True)

        self.mask_token = nn.Parameter(torch.zeros(1, 1, decoder_embed_dim))

        self.decoder_pos_embed = nn.Parameter(
            torch.zeros(1, num_patches + 1, decoder_embed_dim), requires_grad=False
        )  # fixed sin-cos embedding

        self.decoder_blocks = nn.ModuleList(
            [
                Block(
                    decoder_embed_dim,
                    decoder_num_heads,
                    mlp_ratio,
                    qkv_bias=qkv_bias,
                    attn_p=attn_p,
                    proj_p=p,
                )
                for _ in range(decoder_depth)
            ]
        )

        self.decoder_norm = norm_layer(decoder_embed_dim)
        self.decoder_pred = nn.Linear(
            decoder_embed_dim, patch_size**2 * in_channels, bias=True
        )  # decoder to patch
        # --------------------------------------------------------------------------

        self.norm_pix_loss = norm_pix_loss

        self.initialize_weights()

    def forward(self, imgs, stage, mask_ratio=0.75):
        if stage == "pretrain":
            latent, mask, ids_restore = self.forward_encoder(imgs, mask_ratio, stage)
            pred = self.forward_decoder(latent, ids_restore)  # [N, L, p*p*3]
            loss = self.forward_loss(imgs, pred, mask)
            # return loss, pred, mask
            return {"loss": loss, "pred": pred, "mask": mask}

        elif stage == "finetune":
            latent, _, _ = self.forward_encoder(imgs, mask_ratio, stage)
            cls_token = latent[:, 0, :]
            pred_cls = self.cls_head(cls_token)
            # return pred_cls
            return {"pred_cls": pred_cls}

        elif stage == "combine":
            latent, mask, ids_restore = self.forward_encoder(imgs, mask_ratio, stage)
            pred = self.forward_decoder(latent, ids_restore)  # [N, L, p*p*3]
            loss = self.forward_loss(imgs, pred, mask)
            cls_token = latent[:, 0, :]
            pred_cls = self.cls_head(cls_token)
            # return loss, pred, mask, pred_cls
            return {
                "loss": loss,
                "pred": pred,
                "mask": mask,
                "pred_cls": pred_cls,
            }

        else:
            raise ValueError("stage must be one of 'pretrain', 'finetune', 'combine'")

    def classify(self, imgs):
        latent, _, _ = self.forward_encoder(imgs, mask_ratio=0.75, stage="finetune")
        cls_token = latent[:, 0, :]
        pred = self.cls_head(cls_token)
        pred = F.softmax(pred, dim=-1)
        pred = pred.argmax(dim=-1)
        return pred

    def predict(self, imgs):
        return self.classify(imgs)

    def forward_encoder(self, x, mask_ratio, stage):
        # embed patches
        x = self.patch_embed(x)

        # add pos embed w/o cls token
        x = x + self.pos_embed[:, 1:, :]

        # masking: length -> length * mask_ratio
        if stage != "finetune":
            x, mask, ids_restore = self.random_masking(x, mask_ratio)
        else:
            mask = None
            ids_restore = None

        # append cls token
        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # apply Transformer blocks
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        return x, mask, ids_restore

    def forward_decoder(self, x, ids_restore):
        # embed tokens
        x = self.decoder_embed(x)

        # append mask tokens to sequence
        mask_tokens = self.mask_token.repeat(
            x.shape[0], ids_restore.shape[1] + 1 - x.shape[1], 1
        )
        x_ = torch.cat([x[:, 1:, :], mask_tokens], dim=1)  # no cls token
        x_ = torch.gather(
            x_, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2])
        )  # unshuffle
        x = torch.cat([x[:, :1, :], x_], dim=1)  # append cls token

        # add pos embed
        x = x + self.decoder_pos_embed

        # apply Transformer blocks
        for blk in self.decoder_blocks:
            x = blk(x)
        x = self.decoder_norm(x)

        # predictor projection
        x = self.decoder_pred(x)

        # remove cls token
        x = x[:, 1:, :]

        return x

    def forward_loss(self, imgs, pred, mask):
        """
        imgs: [N, 3, H, W]
        pred: [N, L, p*p*3]
        mask: [N, L], 0 is keep, 1 is remove,
        """
        target = self.patchify(imgs)
        if self.norm_pix_loss:
            mean = target.mean(dim=-1, keepdim=True)
            var = target.var(dim=-1, keepdim=True)
            target = (target - mean) / (var + 1.0e-6) ** 0.5

        loss = (pred - target) ** 2
        loss = loss.mean(dim=-1)  # [N, L], mean loss per patch

        loss = (loss * mask).sum() / mask.sum()  # mean loss on removed patches
        return loss

    def initialize_weights(self):
        num_patches = self.patch_embed.get_num_patches()
        pos_embed = get_2d_sincos_pos_embed(
            self.pos_embed.shape[-1],
            int(num_patches**0.5),
            cls_token=True,
        )
        self.pos_embed.data.copy_(torch.from_numpy(pos_embed).float().unsqueeze(0))

        decoder_pos_embed = get_2d_sincos_pos_embed(
            self.decoder_pos_embed.shape[-1],
            int(num_patches**0.5),
            cls_token=True,
        )
        self.decoder_pos_embed.data.copy_(
            torch.from_numpy(decoder_pos_embed).float().unsqueeze(0)
        )

        w = self.patch_embed.projection_layer.weight.data
        torch.nn.init.xavier_uniform_(w.view([w.shape[0], -1]))

        torch.nn.init.normal_(self.cls_token, std=0.02)
        torch.nn.init.normal_(self.mask_token, std=0.02)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            # we use xavier_uniform following official JAX ViT:
            torch.nn.init.xavier_uniform_(m.weight)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def random_masking(self, x, mask_ratio):
        """
        Perform per-sample random masking by per-sample shuffling.
        Per-sample shuffling is done by argsort random noise.
        x: [N, L, D], sequence
        """
        N, L, D = x.shape  # batch, length, dim
        len_keep = int(L * (1 - mask_ratio))

        noise = torch.rand(N, L, device=x.device)  # noise in [0, 1]

        # sort noise for each sample
        ids_shuffle = torch.argsort(
            noise, dim=1
        )  # ascend: small is keep, large is remove
        ids_restore = torch.argsort(ids_shuffle, dim=1)

        # keep the first subset
        ids_keep = ids_shuffle[:, :len_keep]
        x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))

        # generate the binary mask: 0 is keep, 1 is remove
        mask = torch.ones([N, L], device=x.device)
        mask[:, :len_keep] = 0
        # unshuffle to get the binary mask
        mask = torch.gather(mask, dim=1, index=ids_restore)

        return x_masked, mask, ids_restore

    def patchify(self, imgs):
        """
        imgs: (N, 3, H, W)
        x: (N, L, patch_size**2 *3)
        """
        p = self.patch_embed.patch_size
        assert imgs.shape[2] == imgs.shape[3] and imgs.shape[2] % p == 0

        h = w = imgs.shape[2] // p
        x = imgs.reshape(shape=(imgs.shape[0], 3, h, p, w, p))
        x = torch.einsum("nchpwq->nhwpqc", x)
        x = x.reshape(shape=(imgs.shape[0], h * w, p**2 * 3))
        return x

    def unpatchify(self, x):
        """
        x: (N, L, patch_size**2 *3)
        imgs: (N, 3, H, W)
        """
        p = self.patch_embed.patch_size
        h = w = int(x.shape[1] ** 0.5)
        assert h * w == x.shape[1]

        x = x.reshape(shape=(x.shape[0], h, w, p, p, 3))
        x = torch.einsum("nhwpqc->nchpwq", x)
        imgs = x.reshape(shape=(x.shape[0], 3, h * p, h * p))
        return imgs

# Lightning Module

In [ ]:
class VITMaskedAttetionModule(pl.LightningModule):
    
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        config = {
            "img_size": cfg.model.img_size,
            "patch_size": cfg.model.patch_size,
            "in_channels": cfg.model.in_channels,
            "num_classes": cfg.model.num_classes,
            "embedding_dim": cfg.model.embedding_dim,
            "depth": cfg.model.depth,
            "n_heads": cfg.model.n_heads,
            "mlp_ratio": cfg.model.mlp_ratio,
            "qkv_bias": cfg.model.qkv_bias,
            "attn_p": cfg.model.attn_p,
            "p": cfg.model.p,
        }
        self.model = MaskedAutoencoderViT(**config)
        self.save_dir = self.cfg.train.save_dir

    def training_step(self, batch, batch_idx):
        x, y = batch
        if self.cfg.stage == 'pretrain':
            y_hat = self.model(x, mask_ratio=0.5, stage="pretrain")
            loss = y_hat['loss']
        elif self.cfg.stage == 'finetune':
            y_hat = self.model(x, mask_ratio=0.5, stage="finetune")
            loss = F.cross_entropy(y_hat["pred_cls"], y)
        elif self.cfg.stage == 'combine':
            y_hat = self.model(x, mask_ratio=0.5, stage="combine")
            loss = F.cross_entropy(y_hat["pred_cls"], y) + y_hat['loss']
        else:
            raise "No such stage"
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        if self.cfg.stage == 'pretrain':
            y_hat = self.model(x, mask_ratio=0.5, stage="pretrain")
            loss = y_hat['loss']
        elif self.cfg.stage == 'finetune':
            y_hat = self.model(x, mask_ratio=0.5, stage="finetune")
            loss = F.cross_entropy(y_hat["pred_cls"], y)
        elif self.cfg.stage == 'combine':
            y_hat = self.model(x, mask_ratio=0.5, stage="combine")
            loss = F.cross_entropy(y_hat["pred_cls"], y) + y_hat['loss']
        else:
            raise "No such stage"
        self.log('val_loss', loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        # y_hat = self.model(x)
        # loss = F.cross_entropy(y_hat, y)
        # preds = torch.argmax(y_hat, dim=1)
        # acc = (y_hat.argmax(dim=1) == y).float().mean()
        # self.log('test_loss', loss, prog_bar=True, logger=True)
        # self.log('test_acc', acc, prog_bar=True, logger=True)
        # # self.log('test_auc', roc_auc_score(y.cpu(), F.softmax(y_hat.cpu(), dim=1), multi_class='ovo'), prog_bar=True, logger=True)
        # # self.log('test_confusion_matrix', torch.from_numpy(confusion_matrix(y.cpu(), preds.cpu())), prog_bar=True, logger=True)
        # self.log('test_precision', precision_score(y.cpu(), preds.cpu(), average='weighted'), prog_bar=True, logger=True)
        # self.log('test_recall', recall_score(y.cpu(), preds.cpu(), average='weighted'), prog_bar=True, logger=True)
        # self.log('test_f1', f1_score(y.cpu(), preds.cpu(), average='weighted'), prog_bar=True, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.cfg.train.lr, 
                                                        weight_decay=self.cfg.train.weight_decay)
        return optimizer
    
    def configure_callbacks(self):
        early_stop_callback = EarlyStopping(monitor='val_loss', min_delta=0.00, patience=3, verbose=False, mode='min')
        checkpoint_callback = ModelCheckpoint(monitor='val_loss', dirpath=self.save_dir, filename=f'{self.cfg.data.name}-{self.cfg.stage}'+'{epoch:02d}-{val_loss:.2f}', save_top_k=3, mode='min')
        return [early_stop_callback, checkpoint_callback]

# Configuration

In [ ]:
import yaml

with open('configs/ViTMaskedAttention-FER.YAML') as f:
    base_config = yaml.safe_load(f)
base_config

In [ ]:
base_pretrained_config_path = 'configs/base-pretrain.yaml'

# base pretrained config
base_pretrained_config = {k:v for k, v in base_config.items()}
base_pretrained_config['model']['img_size'] = 96
base_pretrained_config['model']['patch_size'] = 32
with open(base_pretrained_config_path, mode='w') as f:
    yaml.dump(base_pretrained_config, f)

In [ ]:
large_pretrained_config_path = 'configs/large-pretrain.yaml'

large_pretrained_config = {k:v for k, v in base_config.items()}
large_pretrained_config['model']['mlp_ratio'] = 4.0
large_pretrained_config['model']['embedding_dim'] = 1024
large_pretrained_config['model']['depth'] = 12
large_pretrained_config['model']['n_heads'] = 8

with open(large_pretrained_config_path, mode='w') as f:
    yaml.dump(large_pretrained_config, f)

# Training


In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import RichProgressBar
from pytorch_lightning.callbacks.progress.rich_progress import RichProgressBarTheme

from omegaconf import OmegaConf, DictConfig

cfg_path = './configs/base-pretrain.YAML'
cfg = OmegaConf.load(cfg_path)

logger = TensorBoardLogger("tb_logs", name="fer")
progress_bar = RichProgressBar()

model = VITMaskedAttetionModule(cfg)
# Load from checkpoint
# model = VITMaskedAttetionModule.load_from_checkpoint("/path/to/checkpoint.ckpt", cfg=cfg)
datamodule = FERDataModule(cfg)

trainer = pl.Trainer(
    accelerator="auto",
    max_epochs=cfg.train.epochs,
    deterministic=True,
    num_sanity_val_steps=0,
    logger=logger,
    callbacks=[progress_bar]
)
trainer.fit(model=model, datamodule=datamodule)

<ipython-input-8-def9c96fe886>:41: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  omega = np.arange(embed_dim // 2, dtype=np.float)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
INFO:pytorch_lightning.acceler

┏━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name  ┃ Type                 ┃ Params ┃
┡━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model │ MaskedAutoencoderViT │ 46.6 M │
└───┴───────┴──────────────────────┴────────┘

Trainable params: 46.6 M                                                                                           
Non-trainable params: 12.8 K                                                                                       
Total params: 46.6 M                                                                                               
Total estimated model params size (MB): 186

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Output()

ValueError: ignored